# Balanceamento da Classe 0 com GANs ClássicosEste notebook reproduz o pipeline definido em `balance_class0_runs.py`, estruturando-o em célulaspara facilitar a experimentação iterativa. O fluxo completo executa as seguintes etapas:1. Treina um DCGAN para a classe 0 do conjunto escolhido do MedMNIST.2. Gera amostras sintéticas para balancear as classes.3. Treina diversas vezes um classificador ResNet-18 sobre o conjunto balanceado.4. Registra métricas e estatísticas agregadas.Configure os parâmetros na seção **Execução do experimento** para repetir os ensaios desejados.

In [2]:
# Importações futuras
from __future__ import annotations

# Importações padrão
import argparse
import json
import os
import random
import time
from dataclasses import dataclass
from pathlib import Path
from typing import Dict, List, Optional, Sequence, Tuple

# Bibliotecas científicas
import numpy as np
import pandas as pd

# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor
from torch.utils.data import ConcatDataset, DataLoader, Dataset, TensorDataset

# TorchVision
from torchvision.models import resnet18

# Módulos personalizados
from classical_gans import DCDiscriminator, DCGenerator, train_gan_for_class
from medmnist_data import load_medmnist_data


In [4]:
# ---------------------------------------------------------------------------
# Constantes e dataclasses auxiliares
# ---------------------------------------------------------------------------

# Métricas de classificação
CLASSIFICATION_METRICS = [
    "acc", "prec", "rec", "f1", "auc", "tn", "fp", "fn", "tp"
]

# Métricas de tempo
TIME_METRICS = [
    "gan_training_time_sec",
    "synthetic_generation_time_sec",
    "classifier_training_time_sec",
    "classifier_eval_time_sec",
]

# Campos numéricos adicionais
ADDITIONAL_NUMERIC_FIELDS = [
    "synthetic_class0_count",
    "balanced_dataset_size",
    "total_real_samples",
    "real_class0_count",
    "real_class1_count",
]

# Campos agregados
AGGREGATION_FIELDS = CLASSIFICATION_METRICS + TIME_METRICS + ADDITIONAL_NUMERIC_FIELDS


# ---------------------------------------------------------------------------
# Dataclasses auxiliares
# ---------------------------------------------------------------------------

from dataclasses import dataclass
from pathlib import Path
from typing import Optional


@dataclass
class ExperimentConfig:
    """Configuração do experimento multietapas."""
    data_flag: str = "breastmnist"
    data_batch_size: int = 128
    latent_dim: int = 100
    gan_epochs: int = 50
    classifier_epochs: int = 5
    num_gan_runs: int = 10
    num_generation_runs: int = 10
    num_classifier_runs: int = 10
    classifier_batch_size: int = 64
    device: Optional[str] = None
    base_seed: int = 2024
    output_dir: Path = Path("balance_class0_runs")


@dataclass
class StageSeeds:
    """Agrupa as sementes de (GAN, geração, classificador) para cada execução."""
    gan_seed: int
    generation_seed: int
    classifier_seed: int


In [5]:
# ---------------------------------------------------------------------------
# Funções utilitárias
# ---------------------------------------------------------------------------

from typing import Optional, Dict
import os
import random
import numpy as np
import torch
from torch import Tensor
from torch.utils.data import Dataset


def _resolve_device(device: Optional[str]) -> torch.device:
    """Seleciona automaticamente o dispositivo (CPU ou GPU)."""
    if device is None or device == "auto":
        return torch.device("cuda" if torch.cuda.is_available() else "cpu")
    return torch.device(device)


def _seed_everything(seed: int) -> None:
    """Define todas as sementes aleatórias para reprodutibilidade."""
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


def _to_scalar(y: Tensor | np.ndarray | int | float) -> int:
    """Converte um tensor, array ou número em escalar inteiro."""
    if isinstance(y, torch.Tensor):
        return int(y.detach().view(-1)[0].item())
    y_np = np.asarray(y)
    return int(y_np.reshape(-1)[0].item())


def count_class_samples(dataset: Dataset) -> Dict[int, int]:
    """Conta o número de amostras por classe em um dataset PyTorch."""
    counts: Dict[int, int] = {}
    for _, label in dataset:
        cls = _to_scalar(label)
        counts[cls] = counts.get(cls, 0) + 1
    return counts


In [6]:
# ---------------------------------------------------------------------------
# Manipulação de dados
# ---------------------------------------------------------------------------

from typing import Sequence, Tuple, List, Dict
import torch
from torch import Tensor, nn
from torch.utils.data import Dataset, TensorDataset, ConcatDataset


def custom_collate_fn(batch: Sequence[Tuple[Tensor, Tensor]]) -> Tuple[Tensor, Tensor]:
    """
    Função de colagem personalizada para DataLoader.
    Concatena imagens e converte rótulos (tensors ou inteiros) em tensores de inteiros.
    """
    images, labels = zip(*batch)
    xs = torch.stack(images, dim=0)
    ys_list: List[int] = []

    for label in labels:
        if isinstance(label, torch.Tensor):
            if label.numel() == 1:
                ys_list.append(int(label.item()))
            else:
                ys_list.append(int(label.argmax().item()))
        else:
            ys_list.append(int(label))

    ys = torch.tensor(ys_list, dtype=torch.long)
    return xs, ys


def build_balanced_dataset(
    train_dataset: Dataset,
    generator: nn.Module,
    *,
    latent_dim: int,
    synthetic_count: int,
    device: torch.device,
    seed: int,
) -> Tuple[Dataset, int]:
    """
    Retorna um dataset com amostras sintéticas adicionais da classe 0.

    Args:
        train_dataset: Dataset real de treino.
        generator: Modelo gerador treinado (GAN).
        latent_dim: Dimensão do vetor latente usado pelo gerador.
        synthetic_count: Quantidade de amostras sintéticas a gerar.
        device: Dispositivo (CPU/GPU) onde o gerador será executado.
        seed: Semente aleatória para reprodutibilidade.

    Returns:
        combined_dataset: Dataset concatenado (real + sintético).
        synthetic_count: Quantidade de amostras sintéticas adicionadas.
    """
    if synthetic_count <= 0:
        return train_dataset, 0

    _seed_everything(seed)
    generator = generator.to(device).eval()

    noise = torch.randn(
        synthetic_count,
        latent_dim,
        1,
        1,
        device=device,
        dtype=torch.float32
    )

    with torch.no_grad():
        synth_imgs = generator(noise)

    if synth_imgs.dim() == 3:
        synth_imgs = synth_imgs.unsqueeze(1)

    synth_imgs = synth_imgs.to(torch.float32).cpu()
    synth_labels = torch.zeros(synthetic_count, dtype=torch.long)

    synthetic_dataset = TensorDataset(synth_imgs, synth_labels)
    combined_dataset = ConcatDataset([train_dataset, synthetic_dataset])

    return combined_dataset, synthetic_count


In [7]:
# ---------------------------------------------------------------------------
# Treinamento
# ---------------------------------------------------------------------------

import time
import numpy as np
from typing import List, Tuple
import torch
import torch.nn.functional as F
from torch import nn, Tensor
from torch.utils.data import DataLoader

from classical_gans import DCGenerator, DCDiscriminator, train_gan_for_class


def train_class0_gan(
    train_loader: DataLoader,
    *,
    latent_dim: int,
    gan_epochs: int,
    device: torch.device,
    seed: int,
    img_channels: int,
) -> Tuple[nn.Module, float]:
    """
    Treina um DCGAN para a classe 0 e retorna o gerador treinado e o tempo de execução.

    Args:
        train_loader: DataLoader com amostras reais da classe 0.
        latent_dim: Dimensão do vetor latente de entrada do gerador.
        gan_epochs: Número de épocas de treino do GAN.
        device: Dispositivo (CPU/GPU).
        seed: Semente aleatória para reprodutibilidade.
        img_channels: Número de canais da imagem (ex: 1 para grayscale, 3 para RGB).

    Returns:
        generator: Modelo gerador treinado.
        elapsed: Tempo total de treinamento (em segundos).
    """
    _seed_everything(seed)
    start = time.time()

    generator = DCGenerator(latent_dim=latent_dim, img_channels=img_channels).to(device)
    discriminator = DCDiscriminator(img_channels=img_channels).to(device)

    generator = train_gan_for_class(
        train_loader=train_loader,
        label_target=0,
        G=generator,
        D=discriminator,
        latent_dim=latent_dim,
        num_epochs=gan_epochs,
        device=device,
    ).eval()

    elapsed = time.time() - start
    return generator, elapsed


def train_classifier(
    model: nn.Module,
    loader: DataLoader,
    *,
    epochs: int,
    device: torch.device,
) -> None:
    """
    Treina um classificador supervisionado usando Cross-Entropy Loss.

    Args:
        model: Rede neural PyTorch.
        loader: DataLoader com os dados de treino.
        epochs: Número de épocas de treinamento.
        device: Dispositivo (CPU/GPU).
    """
    model.to(device)
    model.train()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    for _ in range(epochs):
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            out = model(x)
            loss = F.cross_entropy(out, y)
            loss.backward()
            optimizer.step()


def evaluate_classifier(
    model: nn.Module,
    loader: DataLoader,
    *,
    device: torch.device,
) -> Tuple[float, float, float, float, float, int, int, int, int]:
    """
    Avalia o classificador em um conjunto de dados e calcula métricas de desempenho.

    Args:
        model: Modelo treinado.
        loader: DataLoader com os dados de teste/validação.
        device: Dispositivo (CPU/GPU).

    Returns:
        acc, prec, rec, f1, auc, tn, fp, fn, tp: Métricas de classificação.
    """
    from sklearn.metrics import (
        accuracy_score,
        confusion_matrix,
        f1_score,
        precision_score,
        recall_score,
        roc_auc_score,
    )

    model.to(device)
    model.eval()

    preds: List[Tensor] = []
    labels: List[Tensor] = []

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            out = model(x)
            preds.append(out.argmax(dim=1).cpu())
            labels.append(y)

    y_true = torch.cat(labels).numpy()
    y_pred = torch.cat(preds).numpy()

    def _safe_metric(func, default=np.nan):
        try:
            return float(func(y_true, y_pred))
        except Exception:
            return float(default)

    acc = _safe_metric(accuracy_score)
    prec = _safe_metric(lambda yt, yp: precision_score(yt, yp, zero_division=0))
    rec = _safe_metric(lambda yt, yp: recall_score(yt, yp, zero_division=0))
    f1 = _safe_metric(lambda yt, yp: f1_score(yt, yp, zero_division=0))
    auc = _safe_metric(roc_auc_score, default=np.nan)

    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

    return acc, prec, rec, f1, auc, int(tn), int(fp), int(fn), int(tp)


In [8]:
# ---------------------------------------------------------------------------
# Execução do experimento
# ---------------------------------------------------------------------------

import time
import torch
import pandas as pd
from torch import nn
from torch.utils.data import DataLoader
from typing import Dict, List, Tuple

# ---------------------------------------------------------------------------
# Funções auxiliares
# ---------------------------------------------------------------------------

def compute_stage_seeds(config: ExperimentConfig, gan_id: int, gen_id: int, clf_id: int) -> StageSeeds:
    """
    Gera sementes determinísticas para as três fases do experimento (GAN, geração e classificador).

    Args:
        config: Objeto de configuração principal do experimento.
        gan_id: Índice da execução do GAN.
        gen_id: Índice da execução de geração sintética.
        clf_id: Índice da execução do classificador.

    Returns:
        StageSeeds: Estrutura contendo as sementes de cada fase.
    """
    gan_seed = config.base_seed + gan_id
    generation_seed = config.base_seed + 1000 * gan_id + gen_id
    classifier_seed = config.base_seed + 1_000_000 * gan_id + 1_000 * gen_id + clf_id
    return StageSeeds(
        gan_seed=gan_seed,
        generation_seed=generation_seed,
        classifier_seed=classifier_seed
    )


def prepare_resnet(device: torch.device) -> nn.Module:
    """
    Cria e retorna uma instância de ResNet18 adaptada para imagens monocanais (1 canal).
    """
    model = resnet18(num_classes=2)
    model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
    return model.to(device)


# ---------------------------------------------------------------------------
# Pipeline principal de execução
# ---------------------------------------------------------------------------

def run_balance_experiments(config: ExperimentConfig) -> Tuple[pd.DataFrame, Dict[str, pd.DataFrame], Dict[str, int]]:
    """
    Executa o pipeline completo de balanceamento com GANs e classificadores.

    Retorna:
        - df_results: DataFrame com resultados de todas as execuções.
        - summary_tables: Dicionário com tabelas agregadas.
        - metadata: Dicionário com informações de configuração e execução.
    """
    device = _resolve_device(config.device)

    # Carrega dados MedMNIST
    bundle = load_medmnist_data(
        data_flag=config.data_flag,
        batch_size=config.data_batch_size,
        download=True,
    )

    train_dataset = bundle.train_dataset
    test_dataset = bundle.test_dataset
    train_loader = bundle.train_loader

    counts = count_class_samples(train_dataset)
    real_class0 = counts.get(0, 0)
    real_class1 = counts.get(1, 0)
    deficit = max(0, real_class1 - real_class0)

    results: List[Dict[str, object]] = []
    img_channels = train_dataset[0][0].shape[0]
    total_real_samples = len(train_dataset)

    # Loop principal
    for gan_run in range(1, config.num_gan_runs + 1):
        seeds_for_gan = compute_stage_seeds(config, gan_run, 0, 0)
        generator, gan_time = train_class0_gan(
            train_loader,
            latent_dim=config.latent_dim,
            gan_epochs=config.gan_epochs,
            device=device,
            seed=seeds_for_gan.gan_seed,
            img_channels=img_channels,
        )

        try:
            for gen_run in range(1, config.num_generation_runs + 1):
                seeds_for_generation = compute_stage_seeds(config, gan_run, gen_run, 0)

                gen_start = time.time()
                balanced_dataset, synth_count = build_balanced_dataset(
                    train_dataset,
                    generator,
                    latent_dim=config.latent_dim,
                    synthetic_count=deficit,
                    device=device,
                    seed=seeds_for_generation.generation_seed,
                )
                generation_time = time.time() - gen_start

                for clf_run in range(1, config.num_classifier_runs + 1):
                    seeds = compute_stage_seeds(config, gan_run, gen_run, clf_run)
                    _seed_everything(seeds.classifier_seed)

                    loader = DataLoader(
                        balanced_dataset,
                        batch_size=config.classifier_batch_size,
                        shuffle=True,
                        pin_memory=device.type == "cuda",
                        drop_last=False,
                        collate_fn=custom_collate_fn,
                    )

                    model = prepare_resnet(device)

                    # Treinamento
                    train_start = time.time()
                    train_classifier(
                        model,
                        loader,
                        epochs=config.classifier_epochs,
                        device=device,
                    )
                    train_time = time.time() - train_start

                    # Avaliação
                    eval_loader = DataLoader(
                        test_dataset,
                        batch_size=config.classifier_batch_size,
                        shuffle=False,
                        pin_memory=device.type == "cuda",
                        drop_last=False,
                        collate_fn=custom_collate_fn,
                    )

                    eval_start = time.time()
                    acc, prec, rec, f1, auc, tn, fp, fn, tp = evaluate_classifier(
                        model, eval_loader, device=device
                    )
                    eval_time = time.time() - eval_start

                    # Registro dos resultados
                    result = {
                        "gan_run_id": gan_run,
                        "generation_run_id": gen_run,
                        "classifier_run_id": clf_run,
                        "ratio": 0.0,
                        "acc": acc,
                        "prec": prec,
                        "rec": rec,
                        "f1": f1,
                        "auc": auc,
                        "tn": tn,
                        "fp": fp,
                        "fn": fn,
                        "tp": tp,
                        "synthetic_class0_count": synth_count,
                        "balanced_dataset_size": len(balanced_dataset),
                        "total_real_samples": total_real_samples,
                        "real_class0_count": real_class0,
                        "real_class1_count": real_class1,
                        "gan_training_time_sec": gan_time,
                        "synthetic_generation_time_sec": generation_time,
                        "classifier_training_time_sec": train_time,
                        "classifier_eval_time_sec": eval_time,
                        "gan_seed": seeds_for_gan.gan_seed,
                        "generation_seed": seeds_for_generation.generation_seed,
                        "classifier_seed": seeds.classifier_seed,
                    }

                    results.append(result)

        finally:
            # Libera memória de GPU entre execuções do GAN
            generator.cpu()
            del generator
            if device.type == "cuda":
                torch.cuda.empty_cache()

    # Agregação final de resultados
    df_results = pd.DataFrame(results)
    summary_tables = {
        "overall": aggregate_metrics(df_results, []),
        "by_gan": aggregate_metrics(df_results, ["gan_run_id"]),
        "by_gan_generation": aggregate_metrics(df_results, ["gan_run_id", "generation_run_id"]),
    }

    metadata = {
        "data_flag": config.data_flag,
        "latent_dim": config.latent_dim,
        "gan_epochs": config.gan_epochs,
        "classifier_epochs": config.classifier_epochs,
        "num_gan_runs": config.num_gan_runs,
        "num_generation_runs": config.num_generation_runs,
        "num_classifier_runs": config.num_classifier_runs,
        "classifier_batch_size": config.classifier_batch_size,
        "data_batch_size": config.data_batch_size,
        "device": str(device),
        "base_seed": config.base_seed,
        "real_class0_count": real_class0,
        "real_class1_count": real_class1,
        "synthetic_needed_for_balance": deficit,
        "total_real_samples": total_real_samples,
        "timestamp": time.strftime("%Y-%m-%dT%H:%M:%SZ", time.gmtime()),
    }

    return df_results, summary_tables, metadata


# ---------------------------------------------------------------------------
# Agregação de métricas
# ---------------------------------------------------------------------------

def aggregate_metrics(df: pd.DataFrame, group_cols: List[str]) -> pd.DataFrame:
    """
    Agrega métricas de classificação, tempo e estatísticas por grupo de execuções.

    Args:
        df: DataFrame com resultados completos.
        group_cols: Colunas de agrupamento (ex: ["gan_run_id"]).

    Returns:
        DataFrame com médias e desvios padrão das métricas.
    """
    if df.empty:
        return pd.DataFrame(columns=group_cols + [f"{m}_mean" for m in AGGREGATION_FIELDS])

    grouped = df.groupby(group_cols) if group_cols else [((), df)]
    rows: List[Dict[str, object]] = []

    for key, group in grouped:
        if not isinstance(key, tuple):
            key = (key,)

        row: Dict[str, object] = {}
        for idx, col in enumerate(group_cols):
            row[col] = key[idx]

        row["num_rows"] = len(group)

        for metric in AGGREGATION_FIELDS:
            if metric in group:
                row[f"{metric}_mean"] = float(group[metric].mean())
                row[f"{metric}_std"] = float(group[metric].std(ddof=0))

        rows.append(row)

    return pd.DataFrame(rows)


In [9]:
# ---------------------------------------------------------------------------
# Persistência de resultados
# ---------------------------------------------------------------------------

import json
from pathlib import Path
from typing import Dict
import pandas as pd


def save_results(
    df_results: pd.DataFrame,
    summary_tables: Dict[str, pd.DataFrame],
    metadata: Dict[str, object],
    output_dir: Path,
) -> None:
    """
    Salva os resultados do experimento (tabelas e metadados) em arquivos CSV e JSON.

    Args:
        df_results: DataFrame contendo os resultados detalhados de todas as execuções.
        summary_tables: Dicionário com tabelas agregadas de métricas.
        metadata: Dicionário com informações da configuração e execução do experimento.
        output_dir: Diretório de saída onde os arquivos serão salvos.

    Saídas:
        - balance_class0_results.csv: Resultados completos.
        - summary_*.csv: Tabelas de resumo (overall, por GAN, etc.).
        - metadata.json: Informações descritivas e parâmetros da execução.
    """
    # Cria o diretório de saída, se necessário
    output_dir.mkdir(parents=True, exist_ok=True)

    # Salva resultados completos
    results_path = output_dir / "balance_class0_results.csv"
    df_results.to_csv(results_path, index=False)

    # Salva tabelas agregadas
    for name, df in summary_tables.items():
        df.to_csv(output_dir / f"summary_{name}.csv", index=False)

    # Salva metadados como JSON
    with open(output_dir / "metadata.json", "w", encoding="utf-8") as f:
        json.dump(metadata, f, indent=2, ensure_ascii=False)


## Execução do experimento. Ajuste os hiperparâmetros abaixo conforme necessário. Dependendo dos valores escolhidos(as execuções padrão são 10×10×10), o processo pode ser bastante demorado. Para testes rápidos,reduza os contadores de repetições e épocas.

In [12]:
# ---------------------------------------------------------------------------
# Exemplo de configuração enxuta para testes rápidos
# ---------------------------------------------------------------------------

config = ExperimentConfig(
    data_flag="breastmnist",
    data_batch_size=128,
    latent_dim=100,
    gan_epochs=50,
    classifier_epochs=1,
    num_gan_runs=1,
    num_generation_runs=1,
    num_classifier_runs=1,
    classifier_batch_size=64,
    device="auto",
    base_seed=2024,
    output_dir=Path("balance_class0_runs_demo"),
)

# ---------------------------------------------------------------------------
# Execução opcional do pipeline completo
# ---------------------------------------------------------------------------

# Descomente as linhas abaixo para executar todo o experimento.
# Isso irá:
# 1. Treinar um GAN para a classe 0.
# 2. Gerar amostras sintéticas para balanceamento.
# 3. Treinar e avaliar classificadores.
# 4. Salvar os resultados e metadados no diretório especificado.

df_results, summary_tables, metadata = run_balance_experiments(config)
save_results(df_results, summary_tables, metadata, config.output_dir)


/home/mahlow/anaconda3/envs/my_env/lib/python3.11/site-packages/torch/cuda/__init__.py:141: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


Using downloaded and verified file: /home/mahlow/.medmnist/breastmnist.npz
Using downloaded and verified file: /home/mahlow/.medmnist/breastmnist.npz
Epoch 1/5 - Loss D: 1.350 | Loss G: 0.885
Epoch 2/5 - Loss D: 0.886 | Loss G: 1.125
Epoch 3/5 - Loss D: 0.638 | Loss G: 1.436
Epoch 4/5 - Loss D: 0.536 | Loss G: 1.693
Epoch 5/5 - Loss D: 0.450 | Loss G: 1.923


/tmp/ipykernel_538779/2751922055.py:27: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  ys_list.append(int(label))


In [13]:
df_results

,gan_run_id,generation_run_id,classifier_run_id,ratio,acc,prec,rec,f1,auc,tn,...,total_real_samples,real_class0_count,real_class1_count,gan_training_time_sec,synthetic_generation_time_sec,classifier_training_time_sec,classifier_eval_time_sec,gan_seed,generation_seed,classifier_seed
0,1,1,1,0.0,0.564103,0.739583,0.622807,0.67619,0.513784,17,...,546,147,399,2.440485,0.070227,6.281519,0.206907,2025,3025,1003025


## Utilitário de linha de comando (opcional)As funções abaixo permitem reutilizar o parser de argumentos do script original, casoqueira invocar o notebook via `papermill` ou semelhante.

In [11]:
# ---------------------------------------------------------------------------
# Parser de argumentos para execução programática
# ---------------------------------------------------------------------------

import argparse
from pathlib import Path


def parse_args() -> ExperimentConfig:
    """
    Cria e interpreta argumentos de linha de comando para configurar o experimento.

    Em um notebook, o parse_args([]) evita erros por ausência de argumentos
    e retorna um objeto ExperimentConfig com valores padrão ou definidos manualmente.

    Returns:
        ExperimentConfig: Objeto de configuração preenchido com os parâmetros fornecidos.
    """
    parser = argparse.ArgumentParser(
        description="Run repeated class-0 balancing experiments with DCGAN and ResNet-18",
    )

    # Argumentos principais
    parser.add_argument("--data-flag", default="breastmnist", help="MedMNIST dataset flag")
    parser.add_argument("--data-batch-size", type=int, default=128, help="Batch size for the GAN data loader")
    parser.add_argument("--latent-dim", type=int, default=100, help="Latent dimension for the GAN")
    parser.add_argument("--gan-epochs", type=int, default=50, help="Number of epochs for GAN training")
    parser.add_argument(
        "--classifier-epochs",
        type=int,
        default=5,
        help="Number of epochs for each ResNet-18 training run",
    )
    parser.add_argument("--num-gan-runs", type=int, default=10, help="How many times to retrain the GAN")
    parser.add_argument(
        "--num-generation-runs",
        type=int,
        default=10,
        help="How many synthetic datasets to generate per GAN training",
    )
    parser.add_argument(
        "--num-classifier-runs",
        type=int,
        default=10,
        help="How many classifier trainings per synthetic dataset",
    )
    parser.add_argument(
        "--classifier-batch-size",
        type=int,
        default=64,
        help="Batch size for the ResNet-18 training and evaluation",
    )
    parser.add_argument(
        "--device",
        default="auto",
        help="Device to use (cuda, cpu or auto)",
    )
    parser.add_argument("--base-seed", type=int, default=2024, help="Base seed for reproducibility")
    parser.add_argument(
        "--output-dir",
        type=Path,
        default=Path("balance_class0_runs"),
        help="Directory where CSV and metadata files will be stored",
    )

    # parse_args([]) evita conflitos de CLI em ambientes interativos
    args = parser.parse_args([])

    return ExperimentConfig(
        data_flag=args.data_flag,
        data_batch_size=args.data_batch_size,
        latent_dim=args.latent_dim,
        gan_epochs=args.gan_epochs,
        classifier_epochs=args.classifier_epochs,
        num_gan_runs=args.num_gan_runs,
        num_generation_runs=args.num_generation_runs,
        num_classifier_runs=args.num_classifier_runs,
        classifier_batch_size=args.classifier_batch_size,
        device=args.device,
        base_seed=args.base_seed,
        output_dir=args.output_dir,
    )
